# Dogs vs. Cats

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Upload Kaggle API token

In [0]:
from pathlib import Path
from google.colab import files

kaggle_path = '/root/.kaggle'
kaggle_dir = Path(kaggle_path)
kaggle_api_token_filename = 'kaggle.json'
kaggle_api_token_path = kaggle_dir/kaggle_api_token_filename

# Upload your `kaggle.json` API token file
uploaded = files.upload()
for filename in uploaded.keys():
    print(f'Successfully uploaded file `{filename}` ({len(uploaded[filename])} bytes)')

# Then move `kaggle.json` into the folder where the API expects to find it
! mkdir -p {kaggle_dir} && mv {kaggle_api_token_filename} {kaggle_dir} && chmod 600 {kaggle_api_token_path}

### Download dataset

In [0]:
# Download dataset
! kaggle competitions download -c 'dogs-vs-cats'

### Define folder structure

In [0]:
# Define the root directory
data_dir = '/root/.kaggle/dogs-vs-cats'

# Create the root directory
! mkdir -p {data_dir}

# Define training directory
train_dir = f'{data_dir}/train'
train_dog_dir = f'{train_dir}/dog'
train_cat_dir = f'{train_dir}/cat'

# Define validation directory
val_dir = f'{data_dir}/val'
val_dog_dir = f'{val_dir}/dog'
val_cat_dir = f'{val_dir}/cat'

# Create training directory
! mkdir -p {train_dog_dir}
! mkdir -p {train_cat_dir}

# Create validation directory
! mkdir -p {val_dog_dir}
! mkdir -p {val_cat_dir}

### Unzip train dataset

In [0]:
# Unzip training dataset in the root directory
! unzip train.zip -d {data_dir}

# Print the number of dogs images
print('Number of dogs images')
! ls -1 {train_dir}/dog* | wc -l

# Print the number of cats images
print('Number of cats images')
! ls -1 {train_dir}/cat* | wc -l

### Organize data

In [0]:
import os
import re
import shutil

# Move training images for a good folder structure
files = os.listdir(train_dir)
for filename in files:
    dog_search = re.search('dog', filename)
    cat_search = re.search('cat', filename)
    if dog_search:
        shutil.move(f'{train_dir}/{filename}', train_dog_dir)
    elif cat_search:
        shutil.move(f'{train_dir}/{filename}', train_cat_dir)

In [0]:
# Take some dogs images from training dataset to build a validation dog dataset 
files = os.listdir(train_dog_dir)
for filename in files:
    dog_search = re.search("5\d\d\d", filename)
    if dog_search:
        shutil.move(f'{train_dog_dir}/{filename}', val_dog_dir)
        
# Take some cats images from training dataset to build a validation cat dataset 
files = os.listdir(train_cat_dir)
for filename in files:
    cat_search = re.search('5\d\d\d', filename)
    if cat_search:
        shutil.move(f'{train_cat_dir}/{filename}', val_cat_dir)

In [0]:
# Verify images
print('Training dog dataset directory')
! ls {train_dog_dir} | head -n 5
print('Training cat dataset directory')
! ls {train_cat_dir} | head -n 5

print('Validation dog dataset directory')
! ls {val_dog_dir} | head -n 5
print('Validation cat dataset directory')
! ls {val_cat_dir} | head -n 5

### Transform data

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

from torchvision import datasets, transforms, models

from collections import OrderedDict

# Automatically use CUDA if it's enabled
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Define transforms for the training data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

# Define transforms for the validation data
val_transforms = transforms.Compose([transforms.Resize(255),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

# Pass transforms in here
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
val_data = datasets.ImageFolder(data_dir + '/val', transform=val_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(val_data, batch_size=64)

### Build model

In [0]:
# Define model
model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

# Replace classifier
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

# Define criterion
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.SGD(model.classifier.parameters(), lr=0.004)

model.to(device)

### Train neural network

In [0]:
train_loss = 0

for inputs, labels in trainloader:

    # Move input and label tensors to the GPU
    inputs, labels = inputs.to(device), labels.to(device)

    # Clear the gradients, do this because gradients are accumulated
    optimizer.zero_grad()

    # Make a forward pass through the network to get the outputs
    outputs = model.forward(inputs)

    # Use the logits to calculate the loss
    loss = criterion(outputs, labels)

    # Perform a backward pass through the network to calculate the gradients
    loss.backward()

    # Update the weights
    optimizer.step()

    # Update train loss
    train_loss += loss.item()
    
    # Set model to evaluation mode
    model.eval()

    # Implement the validation pass
    with torch.no_grad():
        
        test_loss = 0
        accuracy = 0

        for inputs, labels in testloader:
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Make a forward pass through the network to get the outputs
            outputs = model.forward(inputs)
                
            # Use the outputs to calculate the loss
            loss = criterion(outputs, labels)
            
            # Update test loss
            test_loss += loss.item()

            # Get the class probabilities
            ps = torch.exp(outputs)

            # Get the most likely class using the ps.topk method
            top_p, top_class = ps.topk(1, dim=1)

            # Check if the predicted classes match the labels
            equals = top_class == labels.view(*top_class.shape)

            # Calculate the percentage of correct predictions
            accuracy += torch.mean(equals.type(torch.FloatTensor))
            
    # Set model back to training mode
    model.train()
    
    # Print out the training loss, testing loss and validation accuracy
    print(f'Train loss: {train_loss/len(trainloader):.3f}.. '
          f'Test loss: {test_loss/len(testloader):.3f}.. '
          f'Test accuracy: {accuracy/len(testloader):.3f}')

In [0]:
print('Our model: \n\n', model, '\n')
print('The state dict keys: \n\n', model.state_dict().keys())

### Save model

In [0]:
# Save model
checkpoint = {
    'parameters' : model.parameters,
    'state_dict' : model.state_dict()
}

torch.save(checkpoint, './dogs-vs-cats.pth')

### Unzip test dataset

In [0]:
# Unzip competition test dataset
! unzip test1.zip -d {data_dir}

### Make predictions

In [0]:
import numpy as np
from PIL import Image

def image_transform(image_path):
    test_transforms = transforms.Compose([transforms.Resize(255),
                                          transforms.CenterCrop(224),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])
    image = Image.open(image_path)
    image_tensor = test_transforms(image)
    return image_tensor

def image_show(image, ax=None, title=None, normalize=True):
    if ax is None:
        fig, ax = plt.subplots()

    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    return ax

In [0]:
import random

# Move model to CPU
device = torch.device('cpu')
model.to(device)

# Define test directory
test_dir = f'{data_dir}/test1'

# Get random image from test dataset
image_path = f'{test_dir}/{random.randint(1, 12500)}.jpg'

# Transform image
image = image_transform(image_path)

# Show image
image_show(image)

# Set model to evaluation mode
model.eval()

# Make prediction
with torch.no_grad():
    image = image[None,:,:,:]
    ps = torch.exp(model(image))
    top_p, top_class = ps.topk(1, dim=1)

if top_class == 1:    
    print(f'Class: dog (confidence: {top_p.item()})')
else:
    print(f'Class: cat (confidence: {top_p.item()})')